<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myStocks_Portfolio_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
import os

# Install es_MX
!/usr/share/locales/install-language-pack en_IN
!dpkg-reconfigure locales

# Restart Python process to pick up the new locales
os.kill(os.getpid(), 9)
'''

'\nimport os\n\n# Install es_MX\n!/usr/share/locales/install-language-pack en_IN\n!dpkg-reconfigure locales\n\n# Restart Python process to pick up the new locales\nos.kill(os.getpid(), 9)\n'

In [2]:
!pip install pyxirr

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from utilities import print_date_time

print_date_time()

gen_output = 0

Run date time (IST): 2024-07-17 20:24:12


In [5]:
from utilities import read_csv

mypfs_df = read_csv('myPortfolioStocks.csv')
mypps_df = read_csv('myProspectsScrips.csv')
myscreen_df = read_csv('myScreenerDB.csv')
myxirr_df = read_csv('myStocks-XIRR.csv')
myxirr_df = read_csv('myStocks-XIRR.csv')
mymmtm_df = read_csv('myProspects-Momentum.csv')

In [6]:
from utilities import get_amt, get_start_end_date, stock_prec_dev

total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()
start_date, end_date = get_start_end_date(-365, 1)

In [11]:
from utilities import get_common_features, get_screener_features, arrange_features, get_portfolio_features, print_portfolio_stats

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

df_screen = get_screener_features(myscreen_df)
# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

df_common_features = get_common_features(df_stocks, start_date, end_date)

df_common_features = pd.merge(df_common_features, df_stocks, on= 'Symbol')
df_common_features = pd.merge(df_common_features, df_screen, on= 'Symbol', how='left')
df_common_features = pd.merge(df_common_features, myxirr_df, on= 'Symbol', how='left')
df_common_features = pd.merge(df_common_features, mymmtm_df, on= 'Symbol', how='left')

#print(df_prec_dev.columns)
df_common_features['Curr_PE'] = round(df_common_features['Close']/df_common_features['EPS'],1)
df_common_features['Dev%_PE'] = round((df_common_features['Curr_PE'] - df_common_features['MedPE'])*100/df_common_features['MedPE'],2)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features, mypf_df)
print_portfolio_stats(df_portfolio_features)

qualified stocks: 80
with latest results: 79
still star stocks: 47
-------------------
Initial Investment:  ₹85,92,755.00
CY Investment:  ₹99,03,755.00
Reserve:  ₹300.00
Current:  ₹1,10,33,595.00
-------------------
Today PnL: ₹25,237.00 (0.23%)
Current PnL: ₹7,54,840.00 (7.62%)
CY Booked + Current PnL: ₹11,29,840.00 (11.41%)
-------------------
Total profit:  ₹10,94,429.00
Total loss:  -₹3,39,593.00
-------------------
Total Booked + Current PnL: ₹24,40,840.00 (28.41%)
Total Booked PnL: ₹16,86,000.00 (19.62%)
Curr Year Booked PnL: ₹3,75,000.00 (3.4%)
Prev Year Booked PnL: ₹13,11,000.00 (15.26%)
Est NTT:  ₹1,50,10,949.00
Est NTT PnL: ₹39,77,354.00 (36.05%)
-------------------
Est LTT:  ₹2,01,83,852.00
Est LTT PnL: ₹91,50,257.00 (82.93%)
Deployed:  ₹86,02,000.00
Current:  ₹1,10,33,595.00
CAGR/XIRR %: 29.16%


In [12]:
# select the features to analyse
cols = ['Symbol', 'Close', 'AvgCost','FTT',  'Dev%_200', 'Dev%_PE', 'Momentum', 'Shares',  'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%','XIRR%', 'CurrAlloc%', 'RRR Ind', 'Criteria', 'Strategy', 'Risk Ind', 'Gained%', 'LatestQtr', 'StarStock', 'Conviction', 'Category']
df_portfolio_features = df_portfolio_features[cols]

if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_portfolio_features.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [13]:
# all portfolio
df_portfolio_features = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
data_table.DataTable(df_portfolio_features, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
18,CAMS,4226.75,2424.02,4450.00,42.75,34.36,UP,25,105669.0,45068.0,...,0.96,8.08,AVR,BTT,71.0,91.20,1,1,M,MISC
63,SHALBY,299.45,264.74,327.00,7.33,NaN,UP,302,90434.0,10483.0,...,0.82,1.26,AVSP,NTT,11.0,63.29,1,1,H,HEALTHCARE
23,EMAMILTD,773.20,489.88,815.00,46.44,29.81,UP,203,156960.0,57514.0,...,1.42,6.77,AVR,BTT,82.0,90.42,1,1,M,FMCG
25,GICRE,417.05,335.06,441.00,28.32,11.79,UP,437,182251.0,35830.0,...,1.65,3.43,AVSP,NTT,40.0,129.93,1,0,M,INSURANCE
49,MARICO,667.35,530.08,725.00,22.23,16.39,UP,200,133470.0,27454.0,...,1.21,2.38,AV40,BTT,31.0,36.18,1,1,H,FMCG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,AAVAS,1788.30,1840.86,3195.80,14.05,-37.47,UP,89,159159.0,-4678.0,...,1.44,-0.04,AVSP,ATH,-4.0,35.93,1,1,M,FINANCE
31,HDFCBANK,1619.75,1569.25,2170.00,6.63,-10.41,NaN,272,440572.0,13736.0,...,3.99,0.09,AV40,BTT,13.0,18.63,1,1,H,BANKS
9,BANDHANBNK,198.43,227.40,400.00,-5.96,-24.18,NaN,827,164102.0,-23958.0,...,1.49,-0.14,AVSP,NTT,-19.0,14.50,1,0,H,BANKS
69,TANLA,1003.05,969.42,2019.39,3.41,-15.72,DOWN,166,166506.0,5582.0,...,1.51,0.03,AAR,ATH,5.0,24.68,1,1,L,IT


In [14]:
# AAR analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AAR') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
65,SONACOMS,714.15,578.05,800.00,16.12,9.37,UP,175,124976.0,23817.0,...,1.13,1.59,AAR,NTT,27.0,40.99,1,1,M,AUTO
52,NEOGEN,1624.75,1489.90,1901.29,8.82,74.22,NaN,68,110483.0,9170.0,...,1.00,0.49,AAR,ATH,9.0,36.05,1,0,L,CHEMICALS
43,JUBLFOOD,585.85,487.35,703.00,14.87,7.00,UP,224,131230.0,22064.0,...,1.19,0.84,AAR,NTT,24.0,36.21,1,0,L,RESTAURANTS
22,DEN,54.44,52.79,75.00,0.81,2.54,UP,1600,87104.0,2640.0,...,0.79,0.08,AAR,NTT,2.0,60.35,1,0,L,ENTERTAINMENT
17,CAMPUS,298.50,297.19,631.85,11.80,8.51,DOWN,197,58804.0,258.0,...,0.53,0.00,AAR,ATH,0.0,39.16,1,0,L,FOOTWEAR
28,HAPPSTMNDS,828.10,919.80,1496.99,-1.53,-12.25,NaN,105,86950.0,-9629.0,...,0.79,-0.14,AAR,ATH,-8.0,11.65,1,1,M,IT
69,TANLA,1003.05,969.42,2019.39,3.41,-15.72,DOWN,166,166506.0,5582.0,...,1.51,0.03,AAR,ATH,5.0,24.68,1,1,L,IT


In [15]:
# candidates to refill
tmp_df = df_portfolio_features[(df_portfolio_features['Current P/L%']>5) & (df_portfolio_features['Current P/L%']<10) & (df_portfolio_features['CurrAlloc%']<2)].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
61,SBIETFPB,264.93,241.24,400.00,9.48,NaN,UP,757,200552.0,17933.0,...,1.82,0.18,AVSP,BTT,18.0,19.29,0,0,H,BANKS
11,BANKINDIA,123.03,112.90,170.00,1.68,-6.08,UP,888,109251.0,8996.0,...,0.99,0.22,AVR,NTT,9.0,58.69,1,1,M,BANKS
37,INDIGOPNTS,1493.45,1407.73,2061.00,4.95,-38.12,UP,124,185188.0,10629.0,...,1.68,0.15,AV40N,NTT,10.0,18.85,1,1,M,PAINTS
24,FINEORG,5221.20,4771.46,7181.93,15.03,5.52,NaN,17,88760.0,7645.0,...,0.80,0.23,AV40N,ATH,8.0,29.40,1,0,M,CHEMICALS
7,BAJAJFINSV,1609.55,1473.47,2116.00,-0.03,-19.33,NaN,78,125545.0,10614.0,...,1.14,0.27,AV40,BTT,11.0,10.30,1,1,H,FINANCE
53,PAGEIND,40953.00,38829.08,53029.39,10.75,2.76,NaN,4,163812.0,8496.0,...,1.48,0.18,AV40,ATH,8.0,23.62,1,1,H,APPARELS
5,ASIANTILES,79.39,75.41,99.00,15.06,-404.32,UP,1244,98761.0,4951.0,...,0.90,0.20,AVR,NTT,5.0,76.23,1,0,M,CERAMICS
38,INDUSINDBK,1445.90,1354.50,1800.00,-2.24,-20.35,NaN,57,82416.0,5210.0,...,0.75,0.26,AVR,NTT,5.0,7.14,1,1,M,BANKS
16,BOROLTD,371.10,342.46,462.00,5.73,NaN,NaN,347,128772.0,9938.0,...,1.17,0.32,AVSP,NTT,10.0,23.58,1,1,M,GLASS
19,CLEAN,1476.20,1389.74,1782.00,5.74,2.05,UP,82,121048.0,7089.0,...,1.10,0.28,AVR,NTT,7.0,15.48,1,0,M,CHEMICALS


In [16]:
# candidates for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['AV40','AV40N',"AVSP"])]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
18,CAMS,4226.75,2424.02,4450.00,42.75,34.36,UP,25,105669.0,45068.0,...,0.96,8.08,AVR,BTT,71.0,91.20,1,1,M,MISC
23,EMAMILTD,773.20,489.88,815.00,46.44,29.81,UP,203,156960.0,57514.0,...,1.42,6.77,AVR,BTT,82.0,90.42,1,1,M,FMCG
65,SONACOMS,714.15,578.05,800.00,16.12,9.37,UP,175,124976.0,23817.0,...,1.13,1.59,AAR,NTT,27.0,40.99,1,1,M,AUTO
52,NEOGEN,1624.75,1489.90,1901.29,8.82,74.22,NaN,68,110483.0,9170.0,...,1.00,0.49,AAR,ATH,9.0,36.05,1,0,L,CHEMICALS
38,INDUSINDBK,1445.90,1354.50,1800.00,-2.24,-20.35,NaN,57,82416.0,5210.0,...,0.75,0.26,AVR,NTT,5.0,7.14,1,1,M,BANKS
51,METROPOLIS,2040.40,1399.27,2345.00,19.53,51.65,UP,67,136707.0,42956.0,...,1.24,2.10,AVR,NTT,57.0,55.73,1,0,M,HEALTHCARE
79,WIPRO,559.70,401.00,611.00,21.67,35.00,UP,421,235634.0,66813.0,...,2.14,3.09,AVR,NTT,85.0,48.28,1,1,M,IT
5,ASIANTILES,79.39,75.41,99.00,15.06,-404.32,UP,1244,98761.0,4951.0,...,0.90,0.20,AVR,NTT,5.0,76.23,1,0,M,CERAMICS
19,CLEAN,1476.20,1389.74,1782.00,5.74,2.05,UP,82,121048.0,7089.0,...,1.10,0.28,AVR,NTT,7.0,15.48,1,0,M,CHEMICALS
29,HATHWAY,24.91,19.74,31.40,13.23,90.33,UP,4000,99640.0,20680.0,...,0.90,0.80,AVR,NTT,24.0,65.51,1,0,M,ENTERTAINMENT


In [17]:
# AVSP analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AVSP') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
63,SHALBY,299.45,264.74,327.0,7.33,NaN,UP,302,90434.0,10483.0,...,0.82,1.26,AVSP,NTT,11.0,63.29,1,1,H,HEALTHCARE
25,GICRE,417.05,335.06,441.0,28.32,11.79,UP,437,182251.0,35830.0,...,1.65,3.43,AVSP,NTT,40.0,129.93,1,0,M,INSURANCE
76,UTIAMC,1107.80,946.90,1207.0,25.74,-8.91,UP,158,175032.0,25422.0,...,1.59,1.62,AVSP,NTT,27.0,49.95,1,1,M,FINANCE
15,BIOCON,353.40,291.09,397.0,27.12,-15.29,UP,516,182354.0,32152.0,...,1.65,1.43,AVSP,NTT,35.0,61.81,1,1,H,PHARMA
59,ROSSARI,821.35,770.50,965.0,8.07,-29.49,UP,190,156056.0,9661.0,...,1.41,0.35,AVSP,NTT,9.0,22.47,1,1,M,CHEMICALS
16,BOROLTD,371.10,342.46,462.0,5.73,NaN,NaN,347,128772.0,9938.0,...,1.17,0.32,AVSP,NTT,10.0,23.58,1,1,M,GLASS
40,IRCTC,1029.00,904.87,1269.0,15.23,6.22,UP,156,160524.0,19364.0,...,1.45,0.52,AVSP,BTT,20.0,67.24,1,1,H,TRAVEL
66,STARHEALTH,589.40,596.56,761.0,5.82,-17.36,NaN,268,157959.0,-1919.0,...,1.43,-0.04,AVSP,NTT,-2.0,18.06,1,1,H,INSURANCE
62,SFL,1034.45,1079.73,1287.0,-1.67,21.48,NaN,187,193442.0,-8468.0,...,1.75,-0.18,AVSP,NTT,-7.0,18.60,1,0,M,MISC
14,BERGEPAINT,520.50,558.54,680.0,-5.54,-31.43,DOWN,317,164998.0,-12059.0,...,1.50,-0.24,AVSP,NTT,-10.0,14.22,1,1,H,PAINTS


In [18]:
# AV40 analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AV40') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
49,MARICO,667.35,530.08,725.00,22.23,16.39,UP,200,133470.0,27454.0,...,1.21,2.38,AV40,BTT,31.0,36.18,1,1,H,FMCG
21,DABUR,640.95,523.35,710.00,16.96,12.00,UP,175,112166.0,20580.0,...,1.02,1.70,AV40,BTT,23.0,29.43,1,1,H,FMCG
54,PIDILITIND,3175.15,2482.94,3483.00,14.92,3.33,UP,58,184159.0,40148.0,...,1.67,2.25,AV40,BTT,47.0,38.33,1,1,H,CHEMICALS
35,ICICIPRULI,654.35,508.03,747.00,17.06,40.70,UP,198,129561.0,28971.0,...,1.17,1.58,AV40,BTT,34.0,36.62,1,0,H,INSURANCE
30,HCLTECH,1569.55,1349.00,1821.00,10.53,32.32,UP,75,117716.0,16541.0,...,1.07,0.88,AV40,BTT,17.0,46.64,1,1,H,IT
39,INFY,1726.05,1436.24,1997.00,14.23,2.98,UP,104,179509.0,30140.0,...,1.63,1.07,AV40,BTT,33.0,32.04,1,1,H,IT
7,BAJAJFINSV,1609.55,1473.47,2116.00,-0.03,-19.33,NaN,78,125545.0,10614.0,...,1.14,0.27,AV40,BTT,11.0,10.30,1,1,H,FINANCE
12,BATAINDIA,1530.00,1550.24,2096.00,2.48,12.18,NaN,83,126990.0,-1680.0,...,1.15,-0.04,AV40,NTT,-2.0,17.82,1,0,H,FOOTWEAR
8,BAJFINANCE,7059.10,7148.10,9150.00,-0.84,-35.23,NaN,23,162359.0,-2047.0,...,1.47,-0.04,AV40,BTT,-2.0,12.41,1,1,H,FINANCE
53,PAGEIND,40953.00,38829.08,53029.39,10.75,2.76,NaN,4,163812.0,8496.0,...,1.48,0.18,AV40,ATH,8.0,23.62,1,1,H,APPARELS


In [19]:
# AV40N analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AV40N') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
48,LUXIND,1973.65,1448.19,2108.00,46.30,85.13,UP,95,187497.0,49919.0,...,1.70,3.91,AV40N,NTT,62.0,83.71,1,0,M,TEXTILES
46,LALPATHLAB,2998.00,2519.90,3689.00,20.21,-0.74,UP,36,107928.0,17212.0,...,0.98,0.69,AV40N,NTT,19.0,49.40,1,1,M,HEALTHCARE
67,SUNTV,806.80,536.47,946.00,22.24,31.89,UP,183,147644.0,49470.0,...,1.34,1.94,AV40N,BTT,68.0,63.29,1,1,M,ENTERTAINMENT
13,BAYERCROP,6597.90,4346.57,8325.00,18.58,15.24,UP,16,105566.0,36021.0,...,0.96,1.30,AV40N,BTT,50.0,55.09,1,1,M,CHEMICALS
24,FINEORG,5221.20,4771.46,7181.93,15.03,5.52,NaN,17,88760.0,7645.0,...,0.80,0.23,AV40N,ATH,8.0,29.40,1,0,M,CHEMICALS
74,TTKPRESTIG,888.00,769.29,1156.08,17.46,25.38,UP,131,116328.0,15551.0,...,1.05,0.44,AV40N,ATH,16.0,32.82,1,0,M,DURABLES
64,SIS,441.30,477.00,678.33,-1.71,73.04,UP,170,75021.0,-6069.0,...,0.68,-0.15,AV40N,ATH,-5.0,9.08,1,0,M,MISC
57,RELAXO,836.05,902.64,1176.00,-2.90,2.51,NaN,128,107014.0,-8524.0,...,0.97,-0.20,AV40N,NTT,-7.0,7.72,1,0,M,FOOTWEAR
71,TATAELXSI,6998.20,7332.28,10499.44,-8.29,-6.65,NaN,14,97975.0,-4677.0,...,0.89,-0.10,AV40N,ATH,-4.0,5.06,1,1,M,IT
44,KANSAINER,279.90,330.73,437.59,-6.70,-65.85,DOWN,362,101324.0,-18400.0,...,0.92,-0.32,AV40N,ATH,-14.0,8.69,1,1,M,PAINTS


In [20]:
# AV200, AVR and AAR analysis
df_tmp = df_portfolio_features[~df_portfolio_features['Criteria'].isin(['AV40','AV40N',"AVSP"])]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,AvgCost,FTT,Dev%_200,Dev%_PE,Momentum,Shares,Current,Current P/L,...,CurrAlloc%,RRR Ind,Criteria,Strategy,Risk Ind,Gained%,LatestQtr,StarStock,Conviction,Category
18,CAMS,4226.75,2424.02,4450.00,42.75,34.36,UP,25,105669.0,45068.0,...,0.96,8.08,AVR,BTT,71.0,91.20,1,1,M,MISC
23,EMAMILTD,773.20,489.88,815.00,46.44,29.81,UP,203,156960.0,57514.0,...,1.42,6.77,AVR,BTT,82.0,90.42,1,1,M,FMCG
65,SONACOMS,714.15,578.05,800.00,16.12,9.37,UP,175,124976.0,23817.0,...,1.13,1.59,AAR,NTT,27.0,40.99,1,1,M,AUTO
52,NEOGEN,1624.75,1489.90,1901.29,8.82,74.22,NaN,68,110483.0,9170.0,...,1.00,0.49,AAR,ATH,9.0,36.05,1,0,L,CHEMICALS
38,INDUSINDBK,1445.90,1354.50,1800.00,-2.24,-20.35,NaN,57,82416.0,5210.0,...,0.75,0.26,AVR,NTT,5.0,7.14,1,1,M,BANKS
51,METROPOLIS,2040.40,1399.27,2345.00,19.53,51.65,UP,67,136707.0,42956.0,...,1.24,2.10,AVR,NTT,57.0,55.73,1,0,M,HEALTHCARE
79,WIPRO,559.70,401.00,611.00,21.67,35.00,UP,421,235634.0,66813.0,...,2.14,3.09,AVR,NTT,85.0,48.28,1,1,M,IT
5,ASIANTILES,79.39,75.41,99.00,15.06,-404.32,UP,1244,98761.0,4951.0,...,0.90,0.20,AVR,NTT,5.0,76.23,1,0,M,CERAMICS
19,CLEAN,1476.20,1389.74,1782.00,5.74,2.05,UP,82,121048.0,7089.0,...,1.10,0.28,AVR,NTT,7.0,15.48,1,0,M,CHEMICALS
29,HATHWAY,24.91,19.74,31.40,13.23,90.33,UP,4000,99640.0,20680.0,...,0.90,0.80,AVR,NTT,24.0,65.51,1,0,M,ENTERTAINMENT
